### Booking Challenge

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch

from typing import List
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm.notebook import tqdm
from torchsummary import summary
from torch.nn.utils.rnn import pad_sequence

In [ ]:
def preprocess_label_df(
        df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    '''
    adds label to the dataset 
    removes the last row for every trip
    removes sequences shorter than 3
    order the df starting from longer sequences
    '''
    df_chunks = []
    df['label'] = df['city_id'].shift(-1)
    idx_labels = np.unique(df.index)

    for idx in tqdm(idx_labels):
        temp_dataset = df.loc[idx].head(-1)
        if type(temp_dataset) == pd.DataFrame and len(temp_dataset) >= 3:
            df_chunks.append(temp_dataset)

    df_chunks = sorted(df_chunks, key=lambda x: len(x), reverse=True)
    new_df = pd.concat(df_chunks)

    return new_df


def front_padding_sequence(tensors: List, num_features: int) -> torch.Tensor:
    """
    Applies front padding to a list of tensors
    """
    sizes = [len(tensor) for tensor in tensors]
    max_size = max(sizes)
    pad_tensors = []
    for q in tensors:
        new_tensor = torch.zeros(max_size, num_features)
        new_tensor[max_size - len(q):] = q
        pad_tensors.append(new_tensor)

    return torch.stack(pad_tensors, dim=0).long()


def custom_padding(batch):
    '''
    pad each batch according to the sequence 
    with the highest length
    '''
    features = [sample[0] for sample in batch]
    features = front_padding_sequence(features, 2)
    labels = [sample[1].unsqueeze(dim=1) for sample in batch]
    labels = front_padding_sequence(labels, 1)

    return features, labels

In [ ]:
df_train = pd.read_csv('/content/sample_data/test_set.csv').set_index('utrip_id')
df_train.index = df_train.index.astype(int)
df_test = pd.read_csv('/content/sample_data/train_set.csv').set_index('utrip_id')
df_test.index = df_test.index.astype(int)

In [ ]:
df_train = preprocess_label_df(df_train)

In [ ]:
df_test = preprocess_label_df(df_test)

### Change utrip_id label

In [ ]:
df_train['label'] = df_train['label'].astype(int)
df_test['label'] = df_test['label'].astype(int)

In [ ]:
utrip_distinct_train = df_train.index.unique()
utrip_train_map = {utrip:i for i, utrip in enumerate(utrip_distinct_train)}
df_train = df_train.rename(index=utrip_train_map)

utrip_distinct_test = df_test.index.unique()
utrip_test_map = {utrip:i for i, utrip in enumerate(utrip_distinct_test)}
df_test = df_test.rename(index=utrip_test_map)

In [ ]:
df_train.head()

,user_id,checkin,checkout,device_class,affiliate_id,booker_country,city_id,hotel_country,label
utrip_id,,,,,,,,,
0,4152137,2016-08-12,2016-08-13,mobile,359,The Devilfire Empire,62394,Novistrana,27189
0,4152137,2016-08-13,2016-08-14,mobile,359,The Devilfire Empire,27189,Novistrana,59713
0,4152137,2016-08-14,2016-08-15,mobile,359,The Devilfire Empire,59713,Novistrana,39027
0,4152137,2016-08-15,2016-08-16,mobile,359,The Devilfire Empire,39027,San Sombrèro,45361
0,4152137,2016-08-16,2016-08-17,mobile,359,The Devilfire Empire,45361,San Sombrèro,47439


#### Create map for data

In [ ]:
df_train.isna().sum()

user_id           0
checkin           0
checkout          0
device_class      0
affiliate_id      0
booker_country    0
city_id           0
hotel_country     1
label             0
dtype: int64

In [ ]:
distinct_affiliate_id = np.unique(df_train.affiliate_id)
affiliate_id_map = {aff_id:i+1 for i, aff_id in enumerate(distinct_affiliate_id)}

distinct_checkin_id = np.unique(df_train.checkin)
checkin_map = {checkin:i+1 for i, checkin in enumerate(distinct_checkin_id)}

distinct_device_class = np.unique(df_train.device_class)
device_map = {device:i+1 for i, device in enumerate(distinct_device_class)}

distinct_city_id = np.unique([df_train.city_id,df_train.label])
city_id_map = {city_id:i+1 for i, city_id in enumerate(distinct_city_id)}

distinct_booker_country = np.unique(df_train.booker_country)
booker_country_map = {booker_country:i+1 for i, booker_country in enumerate(distinct_booker_country)}

distinct_hotel_country = np.unique(df_train.hotel_country.astype(str))
hotel_country_map = {hotel_country:i+1 for i, hotel_country in enumerate(distinct_hotel_country)}

In [ ]:
df_train['checkin'] = df_train['checkin'].map(lambda x: checkin_map.get(x))
df_train['affiliate_id'] = df_train['affiliate_id'].map(lambda x: affiliate_id_map.get(x))
df_train['city_id'] = df_train['city_id'].map(lambda x: city_id_map.get(x))
df_train['label'] = df_train['label'].map(lambda x: city_id_map.get(x))
df_train['booker_country'] = df_train['booker_country'].map(lambda x: booker_country_map.get(x))

In [ ]:
df_test['checkin'] = df_test['checkin'].map(lambda x: checkin_map.get(x, 0))
df_test['affiliate_id'] = df_test['affiliate_id'].map(lambda x: affiliate_id_map.get(x, 0))
df_test['city_id'] = df_test['city_id'].map(lambda x: city_id_map.get(x, 0))
df_test['label'] = df_test['label'].map(lambda x: city_id_map.get(x, 0))
df_test['booker_country'] = df_test['booker_country'].map(lambda x: booker_country_map.get(x, 0))

#### Create a dataset

In [ ]:
class BookingDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return max(self.df.index)
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        city_id = row['city_id'].values
        label_id = row['label'].values
        affiliate_id = row['affiliate_id']
        
        city_id = torch.tensor(row['city_id'].values).view(-1, 1)
        affiliate_id = torch.tensor(row['affiliate_id'].values).view(-1, 1)
        features = torch.cat((city_id, affiliate_id), 1)
        features = features.view(-1, 2)
        
        label_id = torch.tensor(row['label'].values).float()
        
        return features, label_id

In [ ]:
train_ds = BookingDataset(df_train)
train_dl = DataLoader(train_ds, batch_size = 4, shuffle = False, collate_fn=custom_padding)

#### Vanilla RNN

In [ ]:
# Vanilla RNN using nn.RNN
class Vanilla_RNN(nn.Module):
    def __init__(self, emb_size, hidden_size, output_size):
        super(Vanilla_RNN, self).__init__()
        
        self.city_emb = nn.Embedding(len(city_id_map)+1, emb_size, padding_idx=0)
        self.affiliate_emb = nn.Embedding(len(affiliate_id_map)+1, emb_size, padding_idx=0)
        
        self.rnn = nn.RNN(emb_size*2, hidden_size)
        
        # here is our g function from the lecture slides
        # linear layer turning the i-th hidden state into the i-th output
        self.g = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        city_emb = self.city_emb(x[:,:,0])
        affiliate_emb = self.affiliate_emb(x[:,:,1])
        x = torch.cat((city_emb, affiliate_emb), dim=2)
        out, hidden = self.rnn(x)
        out = self.g(out)

        return out, hidden

In [ ]:
v_rnn = Vanilla_RNN(24, 100, 50)

#### LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, emb_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.city_emb = nn.Embedding(len(city_id_map)+1, emb_size, padding_idx=0)
        self.affiliate_emb = nn.Embedding(len(affiliate_id_map)+1, emb_size, padding_idx=0)
        self.lstm = nn.LSTM(emb_size*2, hidden_size, batch_first=True)
        self.g = nn.Linear(hidden_size, len(city_id_map)+1)
        
    def forward(self, x):
        city_emb = self.city_emb(x[:,:,0])
        affiliate_emb = self.affiliate_emb(x[:,:,1])
        x = torch.cat((city_emb, affiliate_emb), dim=2)
        out, hidden = self.lstm(x)
        out = self.g(out) 
        return out, hidden

In [ ]:
lstm = LSTM(50, 100)

In [ ]:
loss_fun = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(lstm.parameters(), lr = 0.01)

In [ ]:
def one_pass(model, dataloader, optimizer, lossFun, backwards=True, print_loss=False):

    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    for x, y in tqdm(dataloader):
        
        output, hidden = model(x)
        output = output.transpose(1,2)
        loss = lossFun(output, y.squeeze(2))
        total_loss += loss.item()
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_loss = total_loss / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
    
    return avg_loss

In [ ]:
one_pass(lstm, train_dl, optimizer, loss_fun)

5.265826428432557

In [ ]:
def one_pass_acc4(model, dataloader, num_points, k=4):
    model.eval()
    total_correct = 0
    for x, y in tqdm(dataloader):
        output, hidden = model(x)
        # print(output.shape)
        # output = output[:,:,2:]
        # print(output.shape)
        result = torch.topk(output, 4, dim=2)
        idx = result.indices
        matches = (torch.eq(idx, y).sum(2)>0).sum(1)>0
        correct = matches.sum().item()
        # print("output shape ", output.shape, "y shape", y.shape)
        # print("Printing output", output)
        # print("Printing y shape", y.shape)
        # print(f"matches = ", matches)
        # print(f"Matches shape = {matches.shape}")
        total_correct += correct
        # print("Correct = ", correct)
        # print(len(y))
        # break
    print(total_correct/num_points)

In [ ]:
one_pass_acc4(lstm, train_dl, len(train_ds))


0.9957118596093971


In [ ]:
test_ds = BookingDataset(df_test)
test_dl = DataLoader(test_ds, batch_size = 4, shuffle = False, collate_fn=custom_padding)
one_pass_acc4(lstm, test_dl, len(test_ds))


0.0007472221325350664


In [ ]:
pd.merge()